In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from Hybrid import *
from Recommenders.SLIM.SLIMElasticNetRecommender import *
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
## SLIM
slim = SLIMElasticNetRecommender(URM_train_validation)
slim.fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668)#, workers=7)
## P3ALPHA
# p3alpha = P3alphaRecommender(URM_train)
# p3alpha.fit(topK=76, alpha=0.377201600381895, normalize_similarity=True)
## RP3BETA
rp3beta = RP3betaRecommender(URM_train_validation)
rp3beta.fit(alpha=0.20026352123406477, beta=0.15999879728761354, topK=32)
## USER KNN
userknn = UserKNNCFRecommender(URM_train_validation)
userknn.fit(topK=469, shrink=38, similarity='asymmetric', normalize=True,
                                       feature_weighting='TF-IDF', asymmetric_alpha=0.40077406933762383)
## ITEM KNN
itemknn = ItemKNNCFRecommender(URM_train_validation)
itemknn.fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                       feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
## EASE_R
ease_r = EASE_R_Recommender(URM_train_validation)
ease_r.fit(topK=24, l2_norm=37.54323189430143)

model = LinearHybridRecommender

from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

SLIMElasticNetRecommender: URM Detected 210 ( 1.7%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 112 ( 0.5%) items with no interactions.
SLIMElasticNetRecommender: Processed 22222 (100.0%) in 4.65 min. Items per second: 79.61
PureSVDRecommender: URM Detected 210 ( 1.7%) users with no interactions.
PureSVDRecommender: URM Detected 112 ( 0.5%) items with no interactions.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 0.71 sec
RP3betaRecommender: URM Detected 210 ( 1.7%) users with no interactions.
RP3betaRecommender: URM Detected 112 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 6681.51 column/sec. Elapsed time 3.33 sec
UserKNNCFRecommender: URM Detected 210 ( 1.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 112 ( 0.5%) items with no interactions.
Similarity column 12638 (100.0%), 9956.21 column/sec. Elapsed time 1.27 sec
ItemKNNCFRecommend

In [3]:
ease = EASE_R_Recommender(URM_train_validation)
ease.fit(topK=24, l2_norm=37.54323189430143)

EASE_R_Recommender: URM Detected 210 ( 1.7%) users with no interactions.
EASE_R_Recommender: URM Detected 112 ( 0.5%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 2.50 min


In [ ]:
from Recommenders.Neural.MultVAERecommender import MultVAERecommender

multvae = MultVAERecommender(URM_train_validation)
multvae.fit(epochs=120, learning_rate=0.0003386225308766206, l2_reg=2.129359930047709e-06, dropout=0.09838599885053126, total_anneal_steps=525579, anneal_cap=0.4033304347929101,batch_size=256)

In [7]:
import itertools
import optuna as op

# Map the names to the recommender objects
# Map the names to the recommender objects
recommender_dict = {
    'slim': slim,
    #'p3alpha': p3alpha,
    'rp3beta': rp3beta,
    'userknn': userknn,
    'itemknn': itemknn,
    'easer': ease,
    #'multvae': multvae
}

# Generate all possible combinations of recommender names
all_combinations = []
for r in range(1, len(recommender_dict) + 1):
    all_combinations.extend(itertools.combinations(recommender_dict.keys(), r))

# Convert combinations to string format
all_combinations_str = ['_'.join(comb) for comb in all_combinations]

def objective(trial):
    # Select a combination from the pre-defined list
    selected_combination_str = trial.suggest_categorical('selected_combination', all_combinations_str)

    # Convert the string back to the original recommender objects
    selected_combination = [recommender_dict[name] for name in selected_combination_str.split('_')]

    # Dynamically generate weights for the selected recommenders
    weights = [trial.suggest_float(f'weight_{i}', 0, 1) for i in range(len(selected_combination))]
    
    # Create hybrid recommender with the suggested weights
    hybrid_recommender = model(URM_train_validation, selected_combination)
    hybrid_recommender.fit(weights)
    
    # Evaluate the hybrid recommender
    result, _ = evaluator_validation.evaluateRecommender(hybrid_recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [ ]:
best_params = {
    "alphas": 1,
}

study_name = "hybrid-allmodels-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db".format(study_name)
#op.delete_study(study_name, storage_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=150)

In [10]:
study.best_params

{'selected_combination': 'slim_rp3beta_itemknn_easer',
 'weight_0': 0.5583799998585726,
 'weight_1': 0.4132791538537288,
 'weight_2': 0.09774430671009315,
 'weight_3': 0.08954789778507188}

In [22]:
hybrid = ScoresHybridRecommender(URM_train_validation, slim, rp3beta, itemknn)
hybrid.fit(alpha=0.5072513954300486, beta=0.30902582824263064)
hybrid_recommender = DifferentLossScoresHybridRecommender(URM_train_validation, hybrid, ease)

ScoresHybridRecommender: URM Detected 210 ( 1.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 112 ( 0.5%) items with no interactions.
DifferentLossScoresHybridRecommender: URM Detected 210 ( 1.7%) users with no interactions.
DifferentLossScoresHybridRecommender: URM Detected 112 ( 0.5%) items with no interactions.


In [23]:
def objective(trial):
    alpha = trial.suggest_float('alpha', 0, 1)
    # Create hybrid recommender with the suggested weights
    hybrid_recommender.fit(norm=2, alpha=alpha)
    
    # Evaluate the hybrid recommender
    result, _ = evaluator_validation.evaluateRecommender(hybrid_recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [ ]:
study_name = "hybrid-slim-rp3-item-ease-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
op.delete_study(study_name, storage_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=150)

In [16]:
study.best_params

{'alpha': 0.8857667100747117,
 'beta': 0.6379807942443021,
 'gamma': 0.0690929184825888,
 'delta': 0.11953478623354052}